# Importing the Data

**Goal**: Classify attacks assuming a multiclass classification system has alerted an attack.

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 

from sklearn.metrics import classification_report, accuracy_score

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Research/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#cleaned data without non-attack values
X = pd.read_csv(root_path+'Data/Features.csv', index_col=0)
Y = pd.read_csv(root_path+'Data/Target.csv', index_col=0)

# Feature Engineering

In [0]:
# %%capture warnings1
# cluster and score
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.cluster import v_measure_score as v

score = []
for i in range(len(X.columns)): # loop number of features
  K = MiniBatchKMeans(n_clusters=len(Y.attack_cat.unique())-1, random_state=0)
  pred = K.fit_predict(X.iloc[:, [i]].values)
  s = v(Y[Y.columns[0]].values,pred)
  score.append(s)
  


In [0]:
# Rank the features and sort

s2 = score
np.asarray(s2)

s1 = []
for i in range(len(X.columns)):
  s1.append(i)
  
np.asarray(s1)

li = list(zip(s1, s2))

def sortSecond(val): 
    return val[1] 
  
li.sort(key = sortSecond, reverse=True) 


print("Index of Feature , adjusted_rand_score\n")
for i in range(len(X.columns)):
    print(li[i])


Index of Feature , adjusted_rand_score

(9, 0.5643840306996547)
(3, 0.5322998900945423)
(18, 0.3432335265189593)
(40, 0.31066840820796954)
(41, 0.3051306545280636)
(37, 0.29834430355843944)
(36, 0.29077316523746394)
(19, 0.28666283661475456)
(42, 0.284981810891032)
(38, 0.2843490870195571)
(1, 0.2758371802253723)
(6, 0.26965238025644755)
(32, 0.2682361292134235)
(13, 0.2670536860868373)
(14, 0.2668434700603014)
(15, 0.2668296304474186)
(12, 0.2626784426259888)
(4, 0.2608142804431214)
(7, 0.25351482290137467)
(22, 0.25032486861702674)
(23, 0.24689591960845766)
(27, 0.2410716835882937)
(39, 0.23949801313751556)
(26, 0.23767964265128905)
(10, 0.23349665663614216)
(28, 0.2292886146178608)
(30, 0.22339783927816825)
(2, 0.22110131552805526)
(29, 0.2080361502027028)
(17, 0.20759993817449932)
(16, 0.20384573732702752)
(8, 0.1683210237103783)
(0, 0.1674318667229353)
(5, 0.15945140696556998)
(11, 0.09156438496079816)
(33, 0.07781006442732108)
(20, 0.07780729553793987)
(21, 0.04997080339916698)
(

In [0]:
# Create a copy of X dataframe with columns sorted by score

titles = []

for i in range(len(X.columns)):
  p = X.columns[li[i][0]]
  titles.append(p)


X1 = pd.DataFrame(columns=titles)

for i in range(len(X.columns)):
  X1[X1.columns[i]] = X[X.columns[li[i][0]]]




In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import accuracy_score

In [0]:
# RF w/out recursive Feature Elimination
x_train, x_test, y_train, y_test = train_test_split(X1, Y, random_state=11)
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred, digits=4))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


                precision    recall  f1-score   support

      Analysis     0.6529    0.1793    0.2814       619
      Backdoor     0.6386    0.0935    0.1631       567
           DoS     0.3141    0.2699    0.2904      4075
      Exploits     0.6356    0.8070    0.7111     11236
       Fuzzers     0.8984    0.8658    0.8818      6127
       Generic     0.9970    0.9861    0.9915     53886
Reconnaissance     0.9240    0.7471    0.8262      3385
     Shellcode     0.6821    0.6082    0.6431       388
         Worms     0.8462    0.2895    0.4314        38

      accuracy                         0.8908     80321
     macro avg     0.7321    0.5385    0.5800     80321
  weighted avg     0.8944    0.8908    0.8881     80321



In [0]:
# %%capture warnings
# Recursive Feature Elemination from # of features to 0 and keep the accuracy score of each

accuracy = []
X2 = X1.copy()

# for i in range(len(X1.columns)-1,-1,-1):
for i in range(len(X1.columns)-1): 
  x_train, x_test, y_train, y_test = train_test_split(X1, Y, test_size=0.2, random_state=11)
  X1.drop(X1.columns[len(X1.columns)-1], axis=1, inplace=True)
  clf = rf(random_state=0, n_jobs=-1)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  accuracy.append(accuracy_score(y_test, y_pred)) 

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff

In [0]:
index = accuracy.index(max(accuracy))
# for i in accuracy:
#   print(i)
  

# print(accuracy)

print(max(accuracy), "max accuracy")

print(index, "index")


X3 = X2.iloc[:,0:len(X.columns)-index]

0.8993105809483791 max accuracy
24 index


# Performance Analysis

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X3, Y, test_size=0.2, random_state=11)

In [0]:
from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
x = list(zip(X3.columns, clf.feature_importances_))

def sortSecond(val): 
    return val[1] 
  
x.sort(key = sortSecond, reverse=True) 

for i in range(len(X3.columns)):
    print(x[i])

('sbytes', 0.33903425644371277)
('ct_dst_ltm', 0.09905313599635612)
('service', 0.09322373512490226)
('smeansz', 0.08660800150341683)
('ct_srv_src', 0.07143712601337812)
('ct_dst_sport_ltm', 0.05815543268179112)
('ct_src_dport_ltm', 0.052876534952698936)
('dmeansz', 0.039052474592705166)
('dbytes', 0.03761332819391243)
('ct_srv_dst', 0.02766243044307428)
('sloss', 0.023134647301493335)
('ct_dst_src_ltm', 0.021475833936096086)
('Dpkts', 0.020728599539763276)
('Spkts', 0.015773564591127054)
('swin', 0.006128745959535435)
('dttl', 0.002985268821840207)
('state', 0.002413808733351392)
('ct_state_ttl', 0.0016821060487092806)
('dwin', 0.0009609691221359572)


In [0]:
clf.score(x_train, y_train) 

0.9218367013453892

In [0]:
print("Accuracy")
clf.score(x_test, y_test) 

Accuracy


0.8993105809483791

In [0]:
y_pred = clf.predict(x_test)

print("Classification\n")
print(classification_report(y_test,y_pred, digits=4))

Classification

                precision    recall  f1-score   support

      Analysis     0.7288    0.1737    0.2806       495
      Backdoor     0.7077    0.0981    0.1723       469
           DoS     0.4007    0.1858    0.2539      3235
      Exploits     0.6258    0.8766    0.7303      8944
       Fuzzers     0.9107    0.8652    0.8874      4846
       Generic     0.9970    0.9886    0.9928     43219
Reconnaissance     0.9129    0.7578    0.8281      2696
     Shellcode     0.7287    0.7196    0.7241       321
         Worms     0.6667    0.6875    0.6769        32

      accuracy                         0.8993     64257
     macro avg     0.7421    0.5948    0.6163     64257
  weighted avg     0.8996    0.8993    0.8912     64257

